In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split

import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from commons.configuration_manager import ConfigurationManager
from src.learning.training.training_transformer import TrainingTransformer
from src.learning.training.label_collector import LabelCollector
from src.learning.training.training_file_reader import TrainingFileReader
from src.learning.models import create_cnn, create_mlp, create_multi_model

config_manager = ConfigurationManager()
config = config_manager.config

reader = TrainingFileReader(path_to_training='../../training/laps/')
transformer = TrainingTransformer(config)
collector = LabelCollector()

In [2]:
filename = 'lap_6_2020_01_24'

df = reader.read_telemetry_as_csv(filename + '.csv')
telemetry = collector.collect_numeric_inputs(df)
diffs = collector.collect_expert_labels(df)

frames = reader.read_training_video(filename + '.avi')
resized_frames = transformer.resize_video_for_training(frames)

print(resized_frames.shape)
print(telemetry.shape)
print(telemetry.columns.values)
print(diffs.shape)
print(diffs.columns.values)

frames_train, frames_test, telemetry_train, telemetry_test, diffs_train, diffs_test = train_test_split(resized_frames, telemetry, diffs, test_size=0.2)

(6578, 40, 60, 3)
(6578, 4)
['cg' 'cs' 'ct' 'cb']
(6578, 4)
['d_gear' 'd_steering' 'd_throttle' 'd_braking']


In [3]:
mlp = create_mlp()
cnn = create_cnn()
multi = create_multi_model(mlp, cnn)
#multi.summary()

epochs = 1
batch_size = 20
verbose = 1

In [4]:
multi.fit([telemetry_train, frames_train], diffs_train,
                validation_data=([telemetry_test, frames_test], diffs_test),
                epochs=epochs,
                batch_size=batch_size,
                verbose=verbose)

W0125 00:39:06.545094 139850352772928 training.py:504] Falling back from v2 loop because of error: Failed to find data adapter that can handle input: (<class 'list'> containing values of types {"<class 'pandas.core.frame.DataFrame'>", "<class 'numpy.ndarray'>"}), <class 'NoneType'>


Train on 5262 samples, validate on 1316 samples
5262/5262 [==============================] - 7s 1ms/sample - loss: 0.2451 - val_loss: 0.1482
